In [ ]:
!unzip /notebooks/storage/ergn8P.zip -d /notebooks
!apt-get update
!apt-get install -y libsm6 libxext6 libxrender-dev
!apt-get install -y ffmpeg
!pip install opencv-python
!pip install requests

# Nhatai

In [1]:
%cd /notebooks
import sys
import os.path
import glob
import re
import cv2
from PIL import Image, ImageDraw, ImageFont
import requests
import shutil
import numpy as np
import torch
import architecture as arch


wrkfol = '/wrk'
encfol = '/enc'
bigrange = range(1,2000)

font = ImageFont.truetype(font='/notebooks/LiberationMono-Bold.ttf', size=90)
xyf = (720, 5)

cropTRS = 500000

def mkENCimg(grp,idx):
  tmp = np.zeros((2160,1,3), np.uint8)
  for ig in grp:
    tmp = np.hstack((ig,tmp))

  rx = 3840-tmp.shape[1]

  if rx > 0:
    talli = np.zeros((2160,rx,3), np.uint8)
    tmp = np.hstack((tmp,talli))
    cv2.imwrite('/enc/%05d.png' % idx, tmp)

  else:
    print('bugRX='+str(rx))


def stig(liss,paII):
  frmcot = len(liss)
  pHT = Image.new('RGB', (2160, 100), (0, 0, 0))
  draw = ImageDraw.Draw(pHT)
  draw.text(xy=xyf, text=paII+'----'+str(frmcot), fill=(255,255,255),font = font)
  HT = np.rot90(np.array(pHT.getdata()).reshape(pHT.size[1], pHT.size[0],3),3)

  imcache=[]
  imcache.append(HT)
  pewrkfol = wrkfol+'/'
  for fi in liss:
    imcache.append(cv2.imread(pewrkfol+fi[:-3]+'png'))

  imcache.append(HT)

  encnu = 1
  wcoting = 0
  tmpgrp = []
  for ig in imcache:
    if(wcoting < 3840):
      tmpgrp.append(ig)
      wcoting+=ig.shape[1]
    else:
      kyp = tmpgrp[-1]
      tmpgrp = tmpgrp[:-1]
      mkENCimg(tmpgrp,encnu)
      tmpgrp = []
      tmpgrp.append(kyp)
      tmpgrp.append(ig)
      wcoting = kyp.shape[1]+ig.shape[1]

      encnu+=1

  mkENCimg(tmpgrp,encnu)

  tmpgrp = []
  imcache = []

  print('Aenc'+paII+' !!   '+str(frmcot))
  !ffmpeg -v quiet -framerate 1 -i /enc/%05d.png -c:v libx264 -crf 30 -pix_fmt yuv420p '/notebooks/storage/mking.mkv'
  shutil.move('/notebooks/storage/mking.mkv','/notebooks/storage/'+paII+'_'+str(encnu)+'.tar')
  print('Benc'+paII+'___END___')

  




def downloadd(url,fnam):
  response = requests.get(url, stream=True)
  if response.status_code == 200:
    with open(wrkfol+fnam, 'wb') as file:
      shutil.copyfileobj(response.raw, file)
    return 1
  else:
    return 0

def EHenty(paInt):
  
  shutil.rmtree(encfol, ignore_errors=True)
  !mkdir {encfol}

  
  shutil.rmtree(wrkfol, ignore_errors=True)
  !mkdir {wrkfol}
  erra = 0
  ppath = 'https://t.nhentai.net/galleries/'+paInt+'/%dt.jpg'
  for i in bigrange:
    if downloadd(ppath % i,'/%05d.jpg' % i) == 0:
      erra+=1
      if erra > 5:
        break
  
  
  flist = os.listdir(wrkfol)
  flist.sort()
  frmcot = len(flist)

  if frmcot == 0:
        return

  suuti = '%05d.jpg' % frmcot
  if suuti != flist[-1]:
    print('somErr? '+paInt+' \nGet='+suuti+' \nReal='+flist[-1])
    
  
  pewrkfol = wrkfol+'/'
  for fi in flist:
    #print(fi)
    to2160(pewrkfol+fi)

  stig(flist,paInt)



  #!ffmpeg -framerate 6 -i {pewrkfol}%05d.png -c:v libx264 -crf 0 -pix_fmt yuv420p '/content/drive/My Drive/ocr/{paInt}_{frmcot}.mkv'
  

    



def mkESRGAN(model_path,scale):

  if not os.path.isfile(model_path):
    model_path = 'storage/'+model_path+'.pth'

  
  model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=scale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
  model.load_state_dict(torch.load(model_path), strict=True)
  model.eval()
  for k, v in model.named_parameters():
      v.requires_grad = False
  return model


def compute_edges(imgB):
    image = ((imgB[0]+imgB[1]+imgB[2])*85).astype(np.uint8)
    _,image = cv2.threshold(image,0,255,cv2.THRESH_OTSU)
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0)
    sobel_x = np.uint8(np.absolute(sobel_x))
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1)
    sobel_y = np.uint8(np.absolute(sobel_y))
    edged =   cv2.bitwise_or(sobel_x, sobel_y)
    
    return edged    

def AutoCrop(imgB):
    edged = compute_edges(imgB)
   
    LLy = edged.shape[0]
    LLx = edged.shape[1]

    

    btest = 6
    while btest < LLy:
      if np.sum(edged[btest,6:-7]) > 0:
        break
      btest+=1
    
    if btest == 6:
        cropUP = btest-6
    else:
        cropUP = btest



    btest = 7
    while btest < LLy:
      if np.sum(edged[LLy-btest,6:-7]) > 0:
        break
      btest+=1
    
    if btest == 7:
        cropDown = LLy+7-btest
    else:
        cropDown = LLy+1-btest

    btest = 6
    while btest < LLx:
      if np.sum(edged[6:-7,btest]) > 0:
        break
      btest+=1
    cropLeft = btest-6


    btest = 7
    while btest < LLx:
      if np.sum(edged[6:-7,LLx-btest]) > 0:
        break
      btest+=1
    cropRight = LLx+7-btest
    
    
    
    if (abs(cropDown-cropUP)*(cropRight-cropLeft))/(LLy*LLx) > 0.3:
      return imgB[:,cropUP:cropDown, cropLeft:cropRight]
    else:
      return imgB

    


def hight2160(innp):
  h = innp.shape[0]
  w = innp.shape[1]
  ratyo = 2160/h
  sgayla = (int(w*ratyo),2160)
  
  if ratyo > 1:
    print('LARGEE===='+str(h))
    return cv2.resize(innp,sgayla, interpolation=cv2.INTER_CUBIC)
  elif ratyo <1:
    return cv2.resize(innp,sgayla, interpolation=cv2.INTER_AREA)
  else:
    return innp
  

def to2160(pathp):
  img = cv2.imread(pathp, cv2.IMREAD_COLOR).astype(np.float32)
  img = img / 255.0
  img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1)))
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  ot1 = model_1st(img_LR).data.squeeze().cpu().numpy()

  #crop
  ot1 = AutoCrop(ot1)
  #crop

  x2w = ot1.shape[1]
  x2h = ot1.shape[2]

  rati = 350000.1/(x2w*x2h)

  if rati < 1:
    rati = rati ** 0.5
    x2w *= rati
    x2h *= rati
    sgayla = (int(x2h),int(x2w))

    rotA = cv2.resize(ot1[0],sgayla, interpolation=cv2.INTER_AREA)
    rotB = cv2.resize(ot1[1],sgayla, interpolation=cv2.INTER_AREA)
    rotC = cv2.resize(ot1[2],sgayla, interpolation=cv2.INTER_AREA)
    ot1 = np.asarray([rotA,rotB,rotC])

  if x2w<x2h:
    ot1 = np.asarray([ np.rot90(ot1[0]),np.rot90(ot1[1]),np.rot90(ot1[2])])


  img = torch.from_numpy(ot1)
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  output = model_2nd(img_LR).data.squeeze().cpu().numpy()
  output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))

  output = hight2160(output)

  output = (output * 255.0).round().clip(0,255.0)
  cv2.imwrite(pathp[:-3]+'png', output)

def countMKVz(pafol):
  flisz = os.listdir(pafol)
  flisz += os.listdir(pafol+'/big')
  mkvlis = []
  tiecot = 0
  for fi in flisz:
    if fi.endswith(".tar"):
      mkvlis.append(fi)
      dimm = re.split('\_|\.',fi)
      tiecot+=int(dimm[1])

  return len(mkvlis), tiecot

  



###
device = torch.device('cuda')
model_1st =  mkESRGAN('FatalimiX',4).to(device)
model_2nd =  mkESRGAN('falcoon300',4).to(device)
###

is400 = False
isOldCrop = False

print('init_fin')

/notebooks
init_finj


In [ ]:
print(is400)


with open('/notebooks/!wkp.txt') as f:
    banki = f.readlines()
banki = [x.strip() for x in banki]

for bbaa in banki:
  EHenty(bbaa)
  if os.path.exists('/notebooks/stopp'):
    break

print('!!!fin!!!')
#4320

False
Aenc7004 !!   30
Benc7004___END___
Aenc7006 !!   26
Benc7006___END___
Aenc7007 !!   23
Benc7007___END___
Aenc7008 !!   74
Benc7008___END___
Aenc7010 !!   58
Benc7010___END___
Aenc7011 !!   58
Benc7011___END___
Aenc7012 !!   54
Benc7012___END___
Aenc7014 !!   14
Benc7014___END___
Aenc7015 !!   38
Benc7015___END___
Aenc7016 !!   38
Benc7016___END___


In [ ]:
print(countMKVz('/content/drive/My Drive/ocr/eh'))
#2700

In [ ]:
print('temp= '+str(countMKVz('/content/drive/My Drive/ocr/eh')))

In [ ]:

mkvparr = '/content/drive/My Drive/ocr/eh'
flist = os.listdir(mkvparr)
flist.sort()
for fi in flist:
  files.download(mkvparr+'/'+fi)
  #aao = input()


# for400

In [2]:

is400 = True


'''
def to2160yoko(path):
  img = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32)
  img = img / 255.0
  img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1)))
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  ot1 = model_1st(img_LR).data.squeeze().cpu().numpy()

  #crop
  ot1 = AutoCrop(ot1)
  #crop

  x2w = ot1.shape[1]
  x2h = ot1.shape[2]

  rati = 600000.1/(x2w*x2h)

  if rati < 1:
    rati = rati ** 0.5
    x2w *= rati
    x2h *= rati
    sgayla = (int(x2h),int(x2w))

    rotA = cv2.resize(ot1[0],sgayla, interpolation=cv2.INTER_AREA)
    rotB = cv2.resize(ot1[1],sgayla, interpolation=cv2.INTER_AREA)
    rotC = cv2.resize(ot1[2],sgayla, interpolation=cv2.INTER_AREA)
    ot1 = np.asarray([rotA,rotB,rotC])

  if x2w>x2h:
    ot1 = np.asarray([ np.rot90(ot1[0],3),np.rot90(ot1[1],3),np.rot90(ot1[2],3)])


  img = torch.from_numpy(ot1)
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  output = model_2nd(img_LR).data.squeeze().cpu().numpy()
  output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))

  output = resizeAndPad(output,(2160,3840))

  output = (output * 255.0).round().clip(0,255.0)
  cv2.imwrite(path[:-3]+'png', output)

'''

def to2160(pathp):
  img = cv2.imread(pathp, cv2.IMREAD_COLOR).astype(np.float32)
  img = img / 255.0
  img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1)))
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  ot1 = model_1st(img_LR).data.squeeze().cpu().numpy()

  #crop
  ot1 = AutoCrop(ot1)
  #crop

  x2w = ot1.shape[1]
  x2h = ot1.shape[2]


  if x2w<x2h:
    ot1 = np.asarray([ np.rot90(ot1[0]),np.rot90(ot1[1]),np.rot90(ot1[2])])


  

  output = ot1
  output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))

  output = hight2160(output)

  output = (output * 255.0).round().clip(0,255.0)
  cv2.imwrite(pathp[:-3]+'png', output)
    
    
print('!!!to400!!!')

!!!to400!!!


In [ ]:
isOldCrop = True

def AutoCrop(imgB):
  avaii = ((imgB[0]+imgB[1]+imgB[2])*85).astype(np.uint8)
  thega,b = cv2.threshold(avaii,0,255,cv2.THRESH_BINARY)
  #display(Image.fromarray(b))

  x=b.shape[1]
  y=b.shape[0]
  edges_x=[]
  edges_y=[]
  for j in range(y):
    for i in range(x):
      if b[j][i]==0:
        edges_x.append(i)
        edges_y.append(j)


  if (len(edges_x) == 0) or (len(edges_y) == 0):
    return imgB

  left=min(edges_x)               #左边界
  right=max(edges_x)              #右边界
  width=right-left                #宽度
  bottom=min(edges_y)             #底部
  top=max(edges_y)                #顶部
  height=top-bottom               #高度

  

  if height*width > 500000:
    return imgB[:,bottom:top,left:right]
  else:
    return imgB

  #display(Image.fromarray((reita[0]*255.0).astype(np.uint8)))

  #print((left,right,width,bottom,top,height))


# temptool

In [ ]:
EHenty('1546591')
#to2160('/wrk/00002.jpg')

In [ ]:
shutil.move('/mking.mkv','/notebooks/zkeep/476_8.tar')

In [ ]:
!unzip /content/44.zip

In [ ]:
!ffmpeg -framerate 6 -i /content/wrk/%05d.png -c:v libx264 -crf 0 -pix_fmt yuv420p '/content/drive/My Drive/ocr/cff420.mkv'

In [ ]:
!ffmpeg -framerate 1 -i /content/enc/%05d.png -c:v libx264 -crf 30 -pix_fmt yuv420p '/content/wkp.mkv'

In [ ]:
shutil.rmtree('/enc')
shutil.rmtree('/wrk')
'''
ffs = os.listdir('/content/wrk')
for filef in ffs:
  if filef.endswith('.png'):
    os.remove('/content/wrk/'+filef)
'''

In [ ]:
for fi in os.listdir('/usr/share/fonts/truetype/liberation'):
  print(fi)

In [ ]:
#!zip -r /notebooks/file.zip /wrk

xflist = os.listdir(wrkfol)
xflist.sort()

print(xflist)

In [ ]:
xflist = os.listdir(wrkfol)
xflist.sort()
flist = []
for fi in xflist:
  if fi.endswith(".jpg"):
    flist.append(fi)




stig(flist,'1384495')